In [30]:
import plotly.express as px

import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.distributions.empirical_distribution import ECDF

from scipy.optimize import minimize
import matplotlib.pyplot as plt

import seaborn as sns



import plotly.express as px
from plotly import io, offline # io to save a html-file of the animation
offline.init_notebook_mode() # offline for better looking interactive graphics
import plotly.figure_factory as ff

import plotly.graph_objects as go  #plotly 4.0.0rc1

import plotly.io as pio
# Set the default renderer to 'colab'
pio.renderers.default = 'colab'

np.random.seed(1)

In [ ]:
# Convert 'Monat' to datetime and extract the year
markets_data['Year'] = pd.to_datetime(markets_data['Monat'], format='%d.%m.%Y').dt.year

# Group by 'Standort' and 'Year', and sum the 'Umsatz'
umsatz_per_standort_year = markt_new_data.groupby(['Standort', 'Year'])['Umsatz'].sum().reset_index()

# Preview the grouped data
umsatz_per_standort_year.head()

,Standort,Year,Umsatz
0,Baar,2022,"2423722819,1532532,131426,6531645,4533287,2288..."
1,Baar,2023,"37538,232379,138878,534492,5537211,137213,2326..."
2,Baar,2024,"31604,0530983,25"
3,Basel,2022,"39970,236720,5548830,1558482,661222,5553588,44..."
4,Basel,2023,"52149,4545560,5558439,0549529,9551258,3548055,..."


In [19]:
markets_data

print(markets_data.head())
print(markets_data['Monat'].dtype)
print(markets_data.info())

# Convert 'Monat' to datetime
markets_data['Monat'] = pd.to_datetime(markets_data['Monat'], format='%d.%m.%Y', errors='coerce')

#Extract the Year
markets_data['Year'] = markets_data['Monat'].dt.year

#Verifying a few Monat and Year rows
print(markets_data[['Monat', 'Year']].head())

print(markets_data[markets_data['Monat'].isna()])

Empty DataFrame
Columns: [Column1, Region, Standort, Monat, Kunden, Umsatz, Früchte und Gemüse, Getränke, Haltbare Grundnahrungsmittel, Kühl-,Molkerei- und Charcuterieprodukte, Material, Near Food, Non Food, Subventionierte Grundnahrungsmittel, Süssigkeiten und Snacks, Tiefkühlprodukte, Year]
Index: []


In [14]:
case_data = pd.read_csv('Falldaten_CaritasbB_updated.csv', delimiter=';')

print(case_data.head())
print(case_data.info())
#
case_data['Datum_Leistung'] = pd.to_datetime(case_data['Datum_Leistung'], errors='coerce')
case_data['Datum_Registrierung_Client'] = pd.to_datetime(case_data['Datum_Registrierung_Client'], errors='coerce')

# Extract the Year
case_data['Leistung_Year'] = case_data['Datum_Leistung'].dt.year
case_data['Registrierung'] = case_data['Datum_Registrierung_Client'].dt.year

# Verify the new columns
print(case_data[['Leistung_Year', 'Registrierung']].head())




   Unnamed: 0 Datum_Leistung    Ereignis_EreignisartID Ereignis_Anzahl  \
0           1     2024-04-19                   Telefon               5   
1           2     2024-04-19                   Telefon               5   
2           3     2024-04-18  Besprechung mit KlientIn              30   
3           4     2024-04-18  Besprechung mit KlientIn              10   
4           5     2024-04-18                   Telefon               5   

   Ereignis_Einheit_ID Datum_Registrierung_Client         Hauptart  \
0                    2                 2019-01-17  02 Finanzbedarf   
1                    2                 2019-01-18  02 Finanzbedarf   
2                    2                 2022-01-12         04 Recht   
3                    2                 2022-02-02      01 Soziales   
4                    2                 2023-09-11  02 Finanzbedarf   

               Materielle_G                         Kulturelle_G  \
0  01 zu geringes Einkommen                             00 keine  

In [32]:

#CODE FOR POLAR GRAPHS OF HAUPTART BY YEAR
# Filter the years first to ensure only 2022, 2023, and 2024 are included
filtered_data = case_data[case_data['Leistung_Year'].isin([2022, 2023, 2024])]

# Group by 'Leistung_Year' and 'Hauptart' and count occurrences
category_counts = filtered_data.groupby(['Leistung_Year', 'Hauptart']).size().reset_index(name='Counts')

filtered_data.info()


# Create a polar bar chart
fig = px.bar_polar(category_counts, r="Counts", theta="Hauptart",
                   color="Leistung_Year", template="plotly",
                   color_discrete_sequence=px.colors.sequential.Plasma_r,
                   title="Category Distribution by Year")
# Show the plot
fig.show()

#Customization
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, category_counts['Counts'].max()]
        )
    ),
    title={'text': "Category Distribution by Year", 'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}
)

fig.show()


<class 'pandas.core.frame.DataFrame'>
Index: 9203 entries, 0 to 9202
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Unnamed: 0                  9203 non-null   int64         
 1   Datum_Leistung              9203 non-null   datetime64[ns]
 2   Ereignis_EreignisartID      9203 non-null   object        
 3   Ereignis_Anzahl             9203 non-null   object        
 4   Ereignis_Einheit_ID         9203 non-null   int64         
 5   Datum_Registrierung_Client  9202 non-null   datetime64[ns]
 6   Hauptart                    9203 non-null   object        
 7   Materielle_G                9203 non-null   object        
 8   Kulturelle_G                9203 non-null   object        
 9   Soziale_G                   9203 non-null   object        
 10  Gesundheitliche_G           9203 non-null   object        
 11  Rechtliche_G                9203 non-null   object        
 1

In [89]:
#CODE FOR BARGRAPHS OF HAUPTART BY YEAR

# Aggregate the counts of each category by year
hauptart_category_counts = case_data[case_data['Leistung_Year'].isin([2022, 2023, 2024])] \
                    .groupby(['Leistung_Year', 'Hauptart']).size() \
                    .reset_index(name='Counts')

# Initialize a figure
hauptart_fig = go.Figure()

# Retrieve unique years and categories
hauptart_years = sorted(hauptart_category_counts['Leistung_Year'].unique())
hauptart_categories = hauptart_category_counts['Hauptart'].unique()

# Create a bar for each year
for year in hauptart_years:
    filtered_data = hauptart_category_counts[hauptart_category_counts['Leistung_Year'] == year]
    hauptart_fig.add_trace(go.Bar(
        x=filtered_data['Hauptart'],
        y=filtered_data['Counts'],
        name=str(year)  # Name the bars as per the year
    ))

# Update the layout for grouped bars
hauptart_fig.update_layout(
    barmode='group',  # This makes bars appear next to each other instead of stacking
    title='Category Distribution by Year',
    xaxis_title='Category',
    yaxis_title='Count',
    legend_title='Year',
    plot_bgcolor='white',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

# Show the plot
hauptart_fig.show()





In [58]:
#Graph for Finanzbedarf by Month

case_data.info()

# Extract year and month from Datum_Leistung
case_data['Leistung_Month'] = case_data['Datum_Leistung'].dt.month

# Filter data for 'Finanzbedarf' and relevant years
finanzbedarf_data = case_data[(case_data['Hauptart'] == '02 Finanzbedarf') & (case_data['Leistung_Year'].isin([2022, 2023]))]

# Aggregate data by year and month
monthly_counts = finanzbedarf_data.groupby(['Leistung_Year', 'Leistung_Month']).size().reset_index(name='Counts')


# Create a line graph
finanz_fig = px.line(monthly_counts,
              x='Leistung_Month',
              y='Counts',
              color='Leistung_Year',
              markers=True,
              labels={'Month': 'Month of the Year', 'Counts': 'Number of Finanzbedarf Cases'},
              title='Monthly Trend of Finanzbedarf Cases by Year')

# Customize the x-axis to show month names instead of numbers
finanz_fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    )
)

# Add custom annotation
finanz_fig.add_annotation(
    x=4,  # April
    y=monthly_counts[(monthly_counts['Leistung_Year'] == 2022) & (monthly_counts['Leistung_Month'] == 4)]['Counts'].values[0],
    text="Start of War in Ukraine",  # Annotation text
    showarrow=True,
    arrowhead=1,
    ax=20,  # adjusts the arrow from point (x, y) by this amount along x and y axis
    ay=-30  # negative moves the text up
)


# Show the plot
finanz_fig.show()


In [83]:
#Finanzbedarf subcategories

case_data.columns

# Filter data for 'Finanzbedarf' and for years 2022 and 2023
finanz_subcat = case_data[
    (case_data['Hauptart'] == '02 Finanzbedarf') &
    (case_data['Leistung_Year'].isin([2022, 2023]))
]

# Ensure 'Materielle_G' is included
print(finanz_subcat[['Leistung_Year', 'Hauptart', 'Materielle_G']].head())


# Group by 'Year' and 'Materielle_G', then count the occurrences
finanz_subcat_yearly_counts = finanz_subcat.groupby(['Leistung_Year', 'Materielle_G']).size().reset_index(name='Counts')


#Create a Bar Graph
graph_finanz_subcat = go.Figure()

# Sort the years for consistent color coding
years = sorted(finanz_subcat_yearly_counts['Leistung_Year'].unique())
colors = ['blue', 'orange']  # Example colors for visual differentiation

# Iterate over each year and add a bar trace for each
for year, color in zip(years, colors):
    filtered_data = finanz_subcat_yearly_counts[finanz_subcat_yearly_counts['Leistung_Year'] == year]
    graph_finanz_subcat.add_trace(go.Bar(
        x=filtered_data['Materielle_G'],
        y=filtered_data['Counts'],
        name=str(year),
        marker_color=color
    ))

# Update the layout to group bars
graph_finanz_subcat.update_layout(
    barmode='group',
    title='Change in Materielle_G Categories for 02 Finanzbedarf by Year',
    xaxis_title="Materielle Goods Categories",
    yaxis_title="Number of Entries",
    legend_title="Year",
    plot_bgcolor='white',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

# Show the plot
graph_finanz_subcat.show()



